##### In this session, we are going to see how we are going to select the best model for production by comparing parameters and metrics. We will also see about the model versioning while changing the parameters. 

> ## Welcome To MLFlow Tracking
---

In [0]:
train_df = spark.read.load("/Volumes/dai/phase2/silver/train_df")
test_df = spark.read.load("/Volumes/dai/phase2/silver/test_df")

In [0]:
display(train_df.head(10))

CustomerID,total_spent,total_transactions,total_quantity,last_purchase_date,is_high_valued,class_weight,features
null,1447682.1199996774,135080,269562,2011-12-09,1,2.418694690265487,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1447682.1199996774"",""135080.0"",""269562.0""]}"
12346,0.0,2,0,2011-01-18,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""2.0"",""0.0""]}"
12347,4309.999999999997,182,2458,2011-12-07,1,2.418694690265487,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""4309.999999999997"",""182.0"",""2458.0""]}"
12348,1797.24,31,2341,2011-09-25,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1797.24"",""31.0"",""2341.0""]}"
12349,1757.55,73,631,2011-11-21,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1757.55"",""73.0"",""631.0""]}"
12350,334.40000000000003,17,197,2011-02-02,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""334.40000000000003"",""17.0"",""197.0""]}"
12352,1545.4100000000005,95,470,2011-11-03,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1545.4100000000005"",""95.0"",""470.0""]}"
12353,89.0,4,20,2011-05-19,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""89.0"",""4.0"",""20.0""]}"
12355,459.4,13,240,2011-05-09,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""459.4"",""13.0"",""240.0""]}"
12356,2811.4300000000007,59,1591,2011-11-17,1,2.418694690265487,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""2811.4300000000007"",""59.0"",""1591.0""]}"


> ##### We have to do the same as we did in previous day, i.e., Converting the features to be trained into one vector. I will use `RandomForest Classifier` here

In [0]:
# from pyspark.ml.feature import VectorAssembler

# assembler = VectorAssembler(
#     inputCols=["total_spent", "total_transactions", "total_quantity"],
#     outputCol="features"
# )

# train_df = assembler.transform(train_df)
# test_df = assembler.transform(test_df)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    labelCol="is_high_valued",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)


In [0]:
import mlflow
import mlflow.spark
from pyspark.ml.classification import RandomForestClassifier

In [0]:
mlflow.set_experiment("/Workspace/Users/kalyanmistcse@gmail.com/Databricks 14 Day AI Challenge - 02/Day - 07")

In [0]:
with mlflow.start_run():
    num_trees = 46
    max_depth = 7

    rf = RandomForestClassifier(
        featuresCol="features",
        labelCol="is_high_valued",
        weightCol="class_weight",
        numTrees=num_trees,
        maxDepth=max_depth
    )

    model = rf.fit(train_df)
    # Make predictions
    predictions = model.transform(test_df)

    # Evaluate
    auc = evaluator.evaluate(predictions)

    # log parameter
    mlflow.log_params({
        'numTrees': num_trees,
        'maxDepth': max_depth
    })

    # log metrics
    mlflow.log_metric("AUC-ROC",auc)

    # log model
    mlflow.spark.log_model(model, "random_forest_model",dfs_tmpdir="/Volumes/dai/phase2/silver/tmp")

#     # log artifact
#     mlflow.log_artifact("/dbfs/FileStore/tables/RF.png")
# # Load model
# loaded_model = mlflow.spark.load_model("runs:/<run_id>/random_forest_model")

print("AUC:", auc)


2026/02/25 14:27:40 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.3.2) contains a local version label (+databricks.connect.17.3.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/02/25 14:27:43 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-b2de8d8d-c84c-4168-ab08-b1/tmpgmdeqpre/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/02/25 14:27:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


AUC: 0.9999429070118346


In [0]:
for i in [12,24,35,19,88]:
    with mlflow.start_run():
        num_trees = i
        max_depth = 7

        rf = RandomForestClassifier(
            featuresCol="features",
            labelCol="is_high_valued",
            weightCol="class_weight",
            numTrees=num_trees,
            maxDepth=max_depth
        )

        model = rf.fit(train_df)
        # Make predictions
        predictions = model.transform(test_df)

        # Evaluate
        auc = evaluator.evaluate(predictions)

        # log parameter
        mlflow.log_params({
            'numTrees': num_trees,
            'maxDepth': max_depth
        })

        # log metrics
        mlflow.log_metric("AUC-ROC",auc)

        # log model
        mlflow.spark.log_model(model, "random_forest_model",dfs_tmpdir="/Volumes/dai/phase2/silver/tmp")

    #     # log artifact
    #     mlflow.log_artifact("/dbfs/FileStore/tables/RF.png")
    # # Load model
    # loaded_model = mlflow.spark.load_model("runs:/<run_id>/random_forest_model")

    print("AUC:", auc)


2026/02/25 14:34:38 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.3.2) contains a local version label (+databricks.connect.17.3.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/02/25 14:34:40 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-b2de8d8d-c84c-4168-ab08-b1/tmpfauc4k7y/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/02/25 14:34:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


AUC: 0.9999429070118346


2026/02/25 14:34:52 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.3.2) contains a local version label (+databricks.connect.17.3.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/02/25 14:34:54 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-b2de8d8d-c84c-4168-ab08-b1/tmp3ocven45/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/02/25 14:34:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


AUC: 0.9999592192941675


2026/02/25 14:35:08 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.3.2) contains a local version label (+databricks.connect.17.3.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/02/25 14:35:10 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-b2de8d8d-c84c-4168-ab08-b1/tmpdsbvjo2b/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/02/25 14:35:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


AUC: 0.9999102824471686


2026/02/25 14:35:25 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.3.2) contains a local version label (+databricks.connect.17.3.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/02/25 14:35:27 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-b2de8d8d-c84c-4168-ab08-b1/tmpie6x7o4e/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/02/25 14:35:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


AUC: 0.9999021263060022


2026/02/25 14:35:41 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.3.2) contains a local version label (+databricks.connect.17.3.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/02/25 14:35:43 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-b2de8d8d-c84c-4168-ab08-b1/tmp5hbjifsh/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/02/25 14:35:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


AUC: 0.9999347508706681


### By comparing the models in the mlflow ui, i came to know that the model with `numTrees` in the range 20-30 gives best `AUC-ROC` score

In [0]:
for i in range(20,30):
    with mlflow.start_run() as run:
        num_trees = i
        max_depth = 7

        rf = RandomForestClassifier(
            featuresCol="features",
            labelCol="is_high_valued",
            weightCol="class_weight",
            numTrees=num_trees,
            maxDepth=max_depth
        )

        model = rf.fit(train_df)
        # Make predictions
        predictions = model.transform(test_df)

        # Evaluate
        auc = evaluator.evaluate(predictions)

        # log parameter
        mlflow.log_params({
            'numTrees': num_trees,
            'maxDepth': max_depth
        })

        # log metrics
        mlflow.log_metric("AUC-ROC",auc)

        # log model
        mlflow.spark.log_model(model, "random_forest_model",dfs_tmpdir="/Volumes/dai/phase2/silver/tmp")

        # Set unique tag for each run
        mlflow.set_tag("unique_run_id", f"rf_{num_trees}_trees_{max_depth}_depth_{run.info.run_id}")

    print("AUC:", auc)

2026/02/25 14:40:56 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.3.2) contains a local version label (+databricks.connect.17.3.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/02/25 14:40:58 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-b2de8d8d-c84c-4168-ab08-b1/tmp2wiwua_6/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/02/25 14:40:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


AUC: 0.9998531894590031


2026/02/25 14:41:12 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.3.2) contains a local version label (+databricks.connect.17.3.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/02/25 14:41:14 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-b2de8d8d-c84c-4168-ab08-b1/tmp6ft644zm/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/02/25 14:41:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


AUC: 0.9999429070118345


2026/02/25 14:41:27 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.3.2) contains a local version label (+databricks.connect.17.3.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/02/25 14:41:29 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-b2de8d8d-c84c-4168-ab08-b1/tmp_zbmkf84/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/02/25 14:41:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

AUC: 0.9998939701648356


2026/02/25 14:43:14 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.3.2) contains a local version label (+databricks.connect.17.3.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/02/25 14:43:16 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-b2de8d8d-c84c-4168-ab08-b1/tmpd_oa1j0k/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/02/25 14:43:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


AUC: 0.9998776578825026


2026/02/25 14:43:31 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.3.2) contains a local version label (+databricks.connect.17.3.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/02/25 14:43:33 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-b2de8d8d-c84c-4168-ab08-b1/tmp9gcc_vxn/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/02/25 14:43:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


AUC: 0.9999347508706681


![image](/Workspace/Users/kalyanmistcse@gmail.com/Databricks 14 Day AI Challenge - 02/Day - 07/image.webp)
---

### As you can see in the above image, with `numTrees` = 24, I am getting the best performace